In [10]:
import requests
import json
from time import sleep
import pandas as pd
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils')
sys.path.append('/home/ndsviriden/mmcc/calc_service/app/web/validation/common')

## API connect and get result

In [11]:
url = 'http://127.0.0.1:5000'
url_roadcast = url+ '/roadcast'
url_calc = url + '/calculation'

#with open("/home/ndsviriden/data_metro_1.txt", "r") as file:
with open("/home/ndsviriden/metro_test.txt", "r") as file:
    data = json.load(file)

for k, v in data['global_forecast'].items():
    data['global_forecast'][k]['cloudiness'] = int(data['global_forecast'][k]['cloudiness'])
    data['global_forecast'][k]['p_weather'] = int(data['global_forecast'][k]['p_weather'])

In [12]:
%%time
with requests.Session() as session:
    session.headers = {'Content-type': 'application/json', 'Host': ".api.mmcc.pkcup.ru", 
                       'cache-control': "no-cache", 'postman-token': '727b926f-c7b5-854e-06d7-bea9df150a12', 
                       'User-Agent':'test'}
    
    response = session.post(url_calc, params={"key":"MeAuthKeyMeSmart"}, json=data)
    #sleep(3) ## What should I do with it
    #print(response.url)
    job_id = json.loads(response.text)['job_id']
    response = session.get(url_roadcast, params={"job_id": job_id, "key": "MeAuthKeyMeSmart"}, timeout=5)
    #print(response.url)
    z = response.content
z

CPU times: user 4.9 ms, sys: 194 µs, total: 5.1 ms
Wall time: 16.1 ms


In [13]:
z

b'{\n  "emergency_report": null, \n  "forecast_alerts": null, \n  "roadcast": null, \n  "roadcast_alerts": null\n}\n'